## 🗺️ Extract Address Information from OLX Listings

Let's scrape the OLX pages again to check if there's address information available on the listing pages.

In [1]:
# Step 1: Load URLs from the CSV file
print("=" * 80)
print("LOADING URLS FROM CSV FILE")
print("=" * 80)

import pandas as pd

csv_path = '../data/sarajevo_flats_olx.csv'

# Load the CSV to get URLs
df_urls = pd.read_csv(csv_path)

if 'url' in df_urls.columns:
    urls_to_check = df_urls['url'].tolist()
    print(f"\n✅ Loaded {len(urls_to_check)} URLs from CSV file")
    print(f"\nSample URLs (first 5):")
    for i, url in enumerate(urls_to_check[:5], 1):
        print(f"  {i}. {url}")
else:
    print("\n❌ No 'url' column found in CSV file")
    urls_to_check = []

print("\n" + "=" * 80)

LOADING URLS FROM CSV FILE

✅ Loaded 1806 URLs from CSV file

Sample URLs (first 5):
  1. https://olx.ba/artikal/71698283
  2. https://olx.ba/artikal/71573692
  3. https://olx.ba/artikal/70756410
  4. https://olx.ba/artikal/71554658
  5. https://olx.ba/artikal/71474528



In [2]:
import time
import random
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service


def clean_text(text):
    """Clean extracted text"""
    if not text:
        return None
    return ' '.join(text.strip().split())


def extract_coordinates_from_google_maps(soup):
    """
    Extract latitude and longitude from Google Maps embedded in the page.
    Works for:
    - ll=LAT,LON
    - @LAT,LON,16z
    - !3dLAT!4dLON
    - JS objects: lat: X, lng: Y
    """
    try:
        # ✅ METHOD 1 — Direct Google Maps links (ll=)
        google_links = soup.find_all("a", href=re.compile(r"google\.com/maps"))
        for link in google_links:
            href = link.get("href", "")

            ll_match = re.search(r"ll=([-\d.]+),([-\d.]+)", href)
            if ll_match:
                lat = float(ll_match.group(1))
                lon = float(ll_match.group(2))
                print("  ✅ Found coords via ll=")
                return lat, lon

        # ✅ METHOD 2 — iframe src (@lat,lon,zoom)
        iframes = soup.find_all("iframe")
        for iframe in iframes:
            src = iframe.get("src", "")

            at_match = re.search(r"@([-\d.]+),([-\d.]+),\d+z", src)
            if at_match:
                lat = float(at_match.group(1))
                lon = float(at_match.group(2))
                print("  ✅ Found coords via @lat,lon")
                return lat, lon

            d_match = re.search(r"!3d([-\d.]+)!4d([-\d.]+)", src)
            if d_match:
                lat = float(d_match.group(1))
                lon = float(d_match.group(2))
                print("  ✅ Found coords via !3d !4d")
                return lat, lon

        # ✅ METHOD 3 — JavaScript objects
        scripts = soup.find_all("script")
        for script in scripts:
            if not script.string:
                continue

            js_match = re.search(
                r"lat[\"']?\s*:\s*([-\d.]+).*?lng[\"']?\s*:\s*([-\d.]+)",
                script.string
            )
            if js_match:
                lat = float(js_match.group(1))
                lon = float(js_match.group(2))
                print("  ✅ Found coords via JS object")
                return lat, lon

        # ✅ METHOD 4 — Raw array format [lat, lon]
        for script in scripts:
            if not script.string:
                continue

            array_match = re.search(r"\[([-\d.]+)\s*,\s*([-\d.]+)\]", script.string)
            if array_match:
                lat = float(array_match.group(1))
                lon = float(array_match.group(2))

                # Optional Sarajevo validation
                if 43.5 < lat < 44.0 and 18.0 < lon < 18.7:
                    print("  ✅ Found coords via raw array")
                    return lat, lon

        return None, None

    except Exception as e:
        print(f"  ⚠️ Coordinate extraction error: {e}")
        return None, None


def extract_address_from_olx(url, driver):
    """
    Extract address info + coordinates from OLX listing
    """
    try:
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, "lxml")

        address_info = {
            "url": url,
            "address": None,
            "location_details": None,
            "map_available": False,
            "latitude": None,
            "longitude": None
        }

        # ✅ COORDINATES
        lat, lon = extract_coordinates_from_google_maps(soup)
        if lat and lon:
            address_info["latitude"] = lat
            address_info["longitude"] = lon
            address_info["map_available"] = True
            print(f"  ✅ Coordinates: {lat}, {lon}")

        # ✅ ADDRESS LABELS
        address_labels = ["Adresa", "Lokacija", "Ulica", "Mjesto"]

        for label in address_labels:
            label_element = soup.find(string=lambda t: t and label in t)
            if label_element:
                parent = label_element.find_parent()
                if parent:
                    value = parent.find_next("h4") or parent.find_next("span") or parent.find_next("div")
                    if value:
                        address_info["address"] = clean_text(value.get_text())
                        address_info["location_details"] = f"Found via '{label}'"
                        break

        # ✅ LOCATION PILL
        if not address_info["address"]:
            location_pill = soup.find("div", class_="btn-pill city")
            if location_pill:
                for svg in location_pill.find_all("svg"):
                    svg.decompose()
                address_info["address"] = clean_text(location_pill.get_text())
                address_info["location_details"] = "Found in location pill"

        # ✅ MAP DETECTION
        if not address_info["map_available"]:
            map_divs = soup.find_all("div", class_=lambda x: x and "map" in x.lower())
            if map_divs:
                address_info["map_available"] = True

        # ✅ META ADDRESS
        if not address_info["address"]:
            meta = soup.find("meta", property="og:street-address")
            if meta and meta.get("content"):
                address_info["address"] = meta["content"]
                address_info["location_details"] = "Found in meta tag"

        return address_info

    except Exception as e:
        print(f"  ❌ Error scraping {url}: {e}")
        return {
            "url": url,
            "address": None,
            "location_details": f"Error: {str(e)}",
            "map_available": False,
            "latitude": None,
            "longitude": None
        }


print("=" * 80)
print("✅ Google Maps Latitude/Longitude Scraper Ready")
print("=" * 80)


✅ Google Maps Latitude/Longitude Scraper Ready


### Test Coordinate Extraction

Let's test the coordinate extraction function with one URL to make sure it works.

In [3]:
import pandas as pd
import time
import random
import os
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.firefox.options import Options


INPUT_CSV = "../data/sarajevo_flats_olx.csv"
OUTPUT_CSV = "../data/sarajevo_flats_olx_with_coordinates.csv"
URL_COLUMN = "url"

MAX_WORKERS = 4      
DELAY_MIN = 1.5
DELAY_MAX = 3.5

csv_lock = threading.Lock()   


df = pd.read_csv(INPUT_CSV)

if URL_COLUMN not in df.columns:
    raise ValueError(f"Column '{URL_COLUMN}' not found in CSV!")



if not os.path.exists(OUTPUT_CSV):
    pd.DataFrame(columns=["url", "latitude", "longitude"])\
        .to_csv(OUTPUT_CSV, index=False)
    print("✅ Created new output CSV")
else:
    print("✅ Output CSV already exists — continuing...")



def create_driver():
    options = Options()
    options.add_argument("--headless")
    return webdriver.Firefox(options=options)


def process_single_url(index, url):
    driver = create_driver()

    try:
        print(f"🔍 [{index}] Processing → {url}")

        result = extract_address_from_olx(url, driver)

        latitude = result.get("latitude")
        longitude = result.get("longitude")

        row = {
            "url": url,
            "latitude": latitude,
            "longitude": longitude
        }


        with csv_lock:
            pd.DataFrame([row]).to_csv(
                OUTPUT_CSV,
                mode="a",
                header=False,
                index=False
            )

        print(f"✅ [{index}] Saved → {latitude}, {longitude}")

    except Exception as e:
        print(f"❌ [{index}] Failed → {e}")

        # ✅ Save failure too
        with csv_lock:
            pd.DataFrame([{
                "url": url,
                "latitude": None,
                "longitude": None
            }]).to_csv(
                OUTPUT_CSV,
                mode="a",
                header=False,
                index=False
            )

    finally:
        driver.quit()

        # ✅ Anti-ban delay PER THREAD
        time.sleep(random.uniform(DELAY_MIN, DELAY_MAX))



print(f"\nStarting multithreaded scraping with {MAX_WORKERS} workers...\n")

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = []

    for index, row in df.iterrows():
        url = row[URL_COLUMN]
        futures.append(
            executor.submit(process_single_url, index + 1, url)
        )

    for future in as_completed(futures):
        pass   # keeps main thread alive


print("\n✅ SCRAPING COMPLETE — ALL THREADS FINISHED")


✅ Created new output CSV

Starting multithreaded scraping with 4 workers...

🔍 [3] Processing → https://olx.ba/artikal/70756410
🔍 [1] Processing → https://olx.ba/artikal/71698283
🔍 [2] Processing → https://olx.ba/artikal/71573692
🔍 [4] Processing → https://olx.ba/artikal/71554658
✅ [1] Saved → None, None
  ✅ Found coords via ll=
  ✅ Coordinates: 43.901206, 18.354117
✅ [3] Saved → 43.901206, 18.354117
  ✅ Found coords via ll=
  ✅ Coordinates: 43.850305, 18.37447
✅ [2] Saved → 43.850305, 18.37447
  ✅ Found coords via ll=
  ✅ Coordinates: 43.714503, 18.288786
✅ [4] Saved → 43.714503, 18.288786
🔍 [5] Processing → https://olx.ba/artikal/71474528
🔍 [6] Processing → https://olx.ba/artikal/71359171
🔍 [7] Processing → https://olx.ba/artikal/41725027
🔍 [8] Processing → https://olx.ba/artikal/71709610
✅ [6] Saved → None, None
✅ [7] Saved → None, None
  ✅ Found coords via ll=
  ✅ Coordinates: 43.800453, 18.246421
✅ [5] Saved → 43.800453, 18.246421
🔍 [9] Processing → https://olx.ba/artikal/71443446

In [14]:
# Load the output CSV with coordinates
df_coords = pd.read_csv("../data/sarajevo_flats_olx.csv")

# Check how many URLs have both latitude and longitude
has_coords = df_coords[['latitude', 'longitude']].notna().all(axis=1).sum()
total_urls = len(df_coords)

print(f"📊 Coordinate Extraction Results:")
print(f"=" * 60)
print(f"Total URLs processed: {total_urls}")
print(f"URLs with coordinates: {has_coords}")
print(f"URLs without coordinates: {total_urls - has_coords}")
print(f"Success rate: {(has_coords / total_urls * 100):.2f}%")
print(f"=" * 60)

# Show some sample data
print(f"\nSample data (first 10 rows):")
print(df_coords.head(10))

📊 Coordinate Extraction Results:
Total URLs processed: 1806
URLs with coordinates: 1175
URLs without coordinates: 631
Success rate: 65.06%

Sample data (first 10 rows):
                                               title  \
0                   Stan Apartman - [Iznajmljivanje]   
1   Stan Sarajevo - Novo Sarajevo - [Iznajmljivanje]   
2   Garaza - Nova Vogosca - 20 m2 - [Iznajmljivanje]   
3  Panorama View Apartman Bjelasnica (Rent) - [Iz...   
4  Stanovi i apartmani Sarajevo - Novi Grad - [Iz...   
5    JAHORINA; prodaja apartmana u novogradnji, 34m2   
6  Garsonjera Sarajevo Sobe Stan {molim čitaj rub...   
7  Dvosoban nenamješten stan u privatnoj kući/ Po...   
8  Stan Sarajevo - Novi Grad 40m2 - [Iznajmljivanje]   
9  Stan Vogošća 15m2 iznajmljivanje - [Iznajmljiv...   

                               url  price_numeric              municipality  \
0  https://olx.ba/artikal/71698283           60.0      Sarajevo - Novi Grad   
1  https://olx.ba/artikal/71573692           80.0  Saraj